In [ ]:
from types import SimpleNamespace
from src.experiment import ExperimentHandler 

#system_cfg = {'system':'bert', 'punct':True, 'action':True, 'context':'back_history', 'context_len':5, 'logging':False}
system_cfg = {'system':'big_bird', 'punct':True, 'action':True, 'debug':False, 
              'mode':'back_history', 'mode_arg':15, 'max_len':512,
              'load':False, 'exp_name':'test', 'save':False
             }

train_cfg = {'lr':1e-5, 'scheduling':False, 'epochs':1, 'print_len':50, 'bsz':8}

system_cfg = SimpleNamespace(**system_cfg)
train_cfg = SimpleNamespace(**train_cfg)

E = ExperimentHandler(system_cfg)
E.train(train_cfg)


100%|██████████| 1003/1003 [00:19<00:00, 52.71it/s]
Attention type 'block_sparse' is not possible if sequence_length: 297 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3.Changing attention type to 'original_full'...


50     2.830    0.287
100    2.301    0.355
150    2.166    0.407
200    2.150    0.448
250    1.700    0.520
300    1.481    0.605
350    1.545    0.608
400    1.612    0.573
450    1.379    0.618
500    1.267    0.655
550    1.069    0.685
600    1.225    0.647
650    1.283    0.647
700    1.148    0.682
750    1.160    0.660
800    1.158    0.693
850    1.198    0.657
900    0.992    0.718
950    1.118    0.677
1000   1.077    0.708
1050   1.037    0.700
1100   1.014    0.713
1150   0.933    0.743
1200   0.981    0.703
1250   0.992    0.715
1300   0.977    0.713
1350   0.935    0.750
1400   1.088    0.685
1450   1.006    0.718
1500   0.954    0.723
1550   0.967    0.757
1600   1.071    0.688
1650   0.953    0.715
1700   1.028    0.710
1750   0.976    0.715
1800   0.962    0.720
1850   0.980    0.743
1900   0.960    0.723
1950   0.817    0.762
2000   0.975    0.723
2050   0.978    0.760
2100   0.968    0.730
2150   0.920    0.733
2200   0.934    0.703
2250   0.919    0.733
2300   0.8

18700  0.590    0.782
18750  0.575    0.805
18800  0.523    0.823


In [ ]:
import numpy as np
import torch
import time

dataset = E.D.dev

confusion_matrix = np.zeros([len(E.D.act_id_dict), len(E.D.act_id_dict)])
for k, batch in enumerate(E.B.batches(dataset), start=1):
    y = E.model(batch.ids, batch.mask)
    loss = E.cross_loss(y, batch.labels)

    y_pred = torch.argmax(y, -1)
    for pred, lab in zip(y_pred, batch.labels):
        confusion_matrix[lab, pred] += 1
        
true_positives = np.diagonal(confusion_matrix)
label_count    = np.sum(confusion_matrix, axis=1)
pred_count     = np.sum(confusion_matrix, axis=0)

precision = true_positives/pred_count
recall = true_positives/label_count

In [ ]:
accuracy = sum(true_positives)/sum(label_count)
print(accuracy)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

sns.set_theme()

rev_dict = {v:k for k,v in E.D.act_id_dict.items()}
act_classes = [E.D.act_names_dict[rev_dict[i]] for i in range(len(rev_dict))]
data = {'index':act_classes, 'precision':precision, 'recall':recall}

df = pd.DataFrame(data)
df = df[df.recall.notna()]
tidy = df[-13:].melt(id_vars='index').rename(columns=str.title)

plt.figure(figsize=(8,8))

ax = sns.barplot(y="Index", 
            x="Value", 
            hue="Variable",
            orient='h',
            data=tidy)

ax.set(xlabel=None)
ax.legend_.set_title(None)